In [ ]:
# from google.colab import auth
# auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# 경로 설정
chat_dir = '/content/gdrive/My Drive/pytest/data/'

In [ ]:
print(chat_dir)

# 1. 형태소 분석

In [ ]:
# 형태소분석기 관련 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk

!pip install JPype1
!pip install rhinoMorph

In [ ]:
# 데이터 읽기 함수 정의
def read_data(filename, encoding='cp949'):
  with open(filename, 'r', encoding=encoding) as f:
    data = [line.split('\t') for line in f.read().splitlines()]
  return data

In [ ]:
# 형태소분석기 준비
import rhinoMorph
rn = rhinoMorph.startRhino()         # 형태소분석기 기동

data = read_data(chat_dir+'data.txt', encoding='cp949')

print('자료 타입:', type(data))
print('전체 문장수:', len(data))
print('형태소 분석 전 모습:', data[:20])

In [ ]:
# data 내용 형태소 분석
with open(chat_dir+'data_morphed.txt', 'w', encoding='utf-8') as f:
    for data_each in data:
        morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each[0], pos=['NNG', 'NNP', 'NP', 'VV', 'VA', 'VX', 'XR', 'IC', 'MM', 'MAG'])
        print("morphed_data_each:", morphed_data_each)
        joined_data_each = ' '.join(morphed_data_each)
        if joined_data_each:
            f.write(joined_data_each + '\t' + data_each[1] + '\n')
    print('Morphological Analysis Completed.')

In [ ]:
print("연결된 마지막 문장: ", joined_data_each)
print("마지막 문장의 라벨: ", data_each[1])

# 2. 훈련데이터와 테스트데이터 분리

In [ ]:
def write_data_list(list, filename, encoding):
    """리스트 변수를 위한 쓰기 함수"""
    with open(chat_dir+filename, 'w') as f:
        for item in list:
            f.write('%s\t%s\n' % (item[0], item[1]))

In [ ]:
# 훈련데이터와 테스트데이터 분리
from sklearn.model_selection import train_test_split

data = read_data(chat_dir+'data_morphed.txt', encoding='utf-8')
train, test = train_test_split(data, test_size=0.2)

write_data_list(list=train, filename='train_data_morphed.txt', encoding='utf-8')
write_data_list(list=test, filename='test_data_morphed.txt', encoding='utf-8')

In [ ]:
# 훈련 데이터 읽기
data = read_data(chat_dir+'train_data_morphed.txt', encoding='utf-8')
print('train length:', len(data))

texts = [line[0] for line in data]                      # 훈련데이터 본문
labels = [line[1] for line in data]                     # 훈련데이터 레이블 부분

In [ ]:
# 테스트 데이터 읽기
data_val = read_data(chat_dir+'test_data_morphed.txt', encoding='utf-8')
print('test length:', len(data_val))

texts_val = [line[0] for line in data_val]          # 테스트 데이터 본문
labels_val = [line[1] for line in data_val]         # 테스트 데이터 label 부분

# 3. 데이터 변환

In [ ]:
# 문자를 숫자로 변환하는 Tokenizing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import math

max_words = 1000                                  # 데이터셋에서 가장 빈도 높은 n개의 단어만 사용한다
maxlen = 20                                       # 각 문장의 길이를 고정시킨다.

tokenizer = Tokenizer(num_words=max_words)        # 상위빈도 1,000 개의 단어만을 추려내는 Tokenizer 객체 생성
tokenizer.fit_on_texts(texts)                     # 단어 인덱스를 구축한다
word_index = tokenizer.word_index                 # 단어 인덱스만 가져온다

print('전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index))
print('word_index type: ', type(word_index))
print('word_index: ',word_index)

In [ ]:
# Tokenizing 결과 확인
# 상위 빈도 1,000(max_words)개의 단어만 word_index의 숫자 리스트로 변환
# Tokenizer 결과가 여기서 반영된다.
data = tokenizer.texts_to_sequences(texts)
print("data:", data)

len_d = [len(d) for d in data]
print("최대 문장 길이: ", max(len_d))
print("최소 문장 길이: ", min(len_d))

In [ ]:
# data 패딩
# 길이를 고정시킨다. maxlen의 수만큼으로 2D 텐서를 만든다. 20을 넘는 데이터는 잘라내고, 모자라는 데이터는 0으로 채운다
data = pad_sequences(data, maxlen=maxlen)

data_val = tokenizer.texts_to_sequences(texts_val)
data_val = pad_sequences(data_val, maxlen=maxlen)

In [ ]:
print(data_val[0])

In [ ]:
print('data type:', type(data))
print('data length:', len(data))
print('texts 0:', texts[0])
print('data 0:', data[0])

In [ ]:
# 원-핫 인코딩 함수
def to_one_hot(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [ ]:
data = to_one_hot(data, dimension=max_words) 
data_val = to_one_hot(data_val, dimension=max_words) 

print('data type:', type(data))
print('data length:', len(data))
print('texts 0:', texts[0])
print('data 0:', data[0])

In [ ]:
# 문자열 label을 숫자형 변수로 치환하는 함수
def labelToIint(labels):
    for count, label in enumerate(labels):
        if label == "배송비":
            labels[count] = 0
        elif label == "담당자문의":
            labels[count] = 1
        elif label == "제품가격":
            labels[count] = 2
        elif label == "배송문의":
            labels[count] = 3
        elif label == "매장코드":
            labels[count] = 4
        elif label == "샘플문의":
            labels[count] = 5
        elif label == "제품불일치":
            labels[count] = 6
        elif label == "반품문의":
            labels[count] = 7
        elif label == "교환문의":
            labels[count] = 8
        elif label == "청구금액":
            labels[count] = 9
    return labels

In [ ]:
# 훈련데이터와 테스트데이터의 label을 숫자로 치환
print("훈련데이터 label 치환 전:\n", labels)
labels = labelToIint(labels)
print("치환 후:", labels)

print("\n테스트데이터 label 치환 전:\n", labels_val)
labels_val = labelToIint(labels_val)
print("치환 후:", labels_val)

In [ ]:
# label을 원-핫 인코딩 한다
class_number = 10                                 # 분류할 클래스의 수

labels = to_one_hot(labels, dimension=class_number)  
print(labels)

labels_val = to_one_hot(labels_val, dimension=class_number)
print(labels_val)

In [ ]:
# Train 데이터와 Test 데이터 준비
print('데이터 텐서의 크기:', data.shape)          
print('레이블 텐서의 크기:', labels.shape)        

x_train = data                         
y_train = labels                    
x_val = data_val
y_val = labels_val

# 4. 모델 구성

In [ ]:
epochs = 5                                        # 수행할 에포크의 수
batch_size = 32                                   # 한 번에 훈련할 배치 사이즈
model_name = 'train_data_morphed.h5'              # 저장될 모델의 이름
tokenizer_name = 'train_data_morphed.pickle'      # 저장될 토크나이저의 이름

In [ ]:
# Define Model
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten

model = Sequential()        # 모델을 새로 정의

model.add(Dense(64, activation='relu', input_shape=(max_words,)))	              # 첫 번째 은닉층
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=class_number, activation='softmax'))

model.summary()

In [ ]:
# Compile Model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
# Train Model
# 32개씩 미니 배치를 만들어 10번의 epoch로 훈련
# 훈련 데이터로 훈련하고, 검증 데이터로 검증한다
# 반환값의 history는 훈련하는 동안 발생한 모든 정보를 담고 있는 딕셔너리이다
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), verbose=1)
history_dict = history.history

In [ ]:
# Save Model
# 만들어진 모델을 이후에 재사용할 수 있도록 저장한다
import pickle

model.save(model_name)

with open(tokenizer_name, 'wb') as file:            # 훈련데이터에서 사용된 상위빈도 1,000개의 단어로 된 Tokenizer 저장(같은 단어를 추출하게 한다)
    pickle.dump(tokenizer, file, protocol=pickle.HIGHEST_PROTOCOL)

# 5. 모델 성능 확인

In [ ]:
# Accuracy & Loss
# history 딕셔너리 안에 있는 정확도와 손실값을 가져와 본다
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

print('Validation accuracy of each epoch:', np.round(val_acc, 3))
epochs = range(1, len(val_acc) + 1)

In [ ]:
# Plotting Performance
import matplotlib.pyplot as plt

plt.plot(epochs, acc, 'bo', label='Training Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training and Validation accuracy')
plt.legend(loc=2)

In [ ]:
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation loss')
plt.legend(loc=2)
plt.show()

In [ ]:
# Load Model
from tensorflow.keras.models import load_model

loaded_model = load_model(model_name)

with open(tokenizer_name, 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

In [ ]:
# 라벨을 답변으로 치환하는 함수
def intToLabel(label_int):
    labels = ''
    if label_int == 0:
        labels = '20,000원 이상 주문하시면 배송비가 없습니다'
    elif label_int == 1:
        labels = '담당자는 홈페이지에 직원의 이름 또는 사번을 넣으시면 자세한 정보를 알 수 있습니다'
    elif label_int == 2:
        labels = '공급가와 소비자가는 홈페이지 > 직원 ID 로그인 > 물품명 > 가격조회 에서 확인 가능합니다'
    elif label_int == 3:
        labels = '배송에는 보통 2일이 소요되며, 빠른 배송을 선택하시면 1일 안에 책임배달합니다'
    elif label_int == 4:
        labels = '매장 코드는 홈페이지 > 매장정보 에서 확인 가능합니다'
    elif label_int == 5:
        labels = '샘플신청은 홈페이지 > 직원 ID 로그인 > 물품명 > 샘플신청 에서 가능합니다'
    elif label_int == 6:
        labels = '다른 제품이 배송되어 죄송합니다. 홈페이지 > 물품명 > 환불신청 또는 전화 상담 부탁드립니다'
    elif label_int == 7:
        labels = '배송받으신 모든 제품은 7일 안에는 반품이 가능합니다. 홈페이지 > 물품명 > 환불신청 에서 반품 가능합니다'
    elif label_int == 8:
        labels = '배송받으신 모든 제품은 7일 안에는 교환이 가능합니다. 홈페이지 > 물품명 > 교환신청 에서 교환 가능합니다'
    elif label_int == 9:
        labels = '청구금액은 매달 12일 이후에 홈페이지 > 직원 ID 로그인 > 청구금액에서 확인 가능합니다'
    return labels

In [ ]:
# 1문장 예측
user_input = input("내용을 입력하세요: ")
morphed_input, poses = rhinoMorph.wholeResult_list(rn, user_input, pos=['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'VCN', 'MAG', 'MAJ', 'IC', 'JKV', 'EF', 'SF'])
text = [morphed_input]

data = loaded_tokenizer.texts_to_sequences(text)
data = pad_sequences(data, maxlen=maxlen)
x_test = to_one_hot(data, dimension=max_words)

predictions = loaded_model.predict(x_test)
label_int = np.argmax(predictions)
label = intToLabel(label_int)
print(label)

In [ ]:
predictions